In [18]:
from dataclasses import dataclass, field
from typing import Optional
from datetime import datetime, timedelta
import yfinance as yf
from fredapi import Fred
from nelson_siegel_svensson.calibrate import calibrate_ns_ols
import pandas as pd
import numpy as np

fred = Fred(api_key='5079f41d061a4037d81f3da69e018803')

## Engine : Yield curve interpolation for cash flow discounting
## 

In [78]:
frequency_mapping_dict = {
    'Annually': 1,
    'Semi-Annually': 2,
    'Quarterly': 4
}

@dataclass 
class Stock:
    Ticker: str

    def price(self):
        return yf.Ticker(self.Ticker).fast_info['lastPrice']

@dataclass
class Interest_Rates:
    Maturity: str
    Frequency: str

    Num_Frequency: int = field(init=False)
    T: int = field(init=False)

    def __post_init__(self):
        try:
            self.Num_Frequency = frequency_mapping_dict[self.Frequency]
        except KeyError:
            raise ValueError(f'Incorrect Frequency: {self.Frequency} should be one of {list(frequency_mapping_dict.keys())}')

        try:
            self.T = (datetime.strptime(self.Maturity, '%Y-%m-%d')-datetime.today()).days/365
        except ValueError:
            raise ValueError(f'Incorrect date : {self.Maturity}. Please use YYYY-MM-DD format')

    @staticmethod
    def _most_recent_business_day():
        today = datetime.today().date()
        while today.weekday() >= 5: 
            today -= timedelta(days=1)
        previous_bd = today-timedelta(days=1)
        data = pd.Series()
        while len(data)==0:
            data = fred.get_series('DGS1', observation_start= previous_bd, observation_end = today)
            previous_bd -= timedelta(days=1)
        
        return today.isoformat(), previous_bd.isoformat()


    @staticmethod
    def _get_yield_data(series_id):
        end_date, start_date = Interest_Rates._most_recent_business_day()
        data = fred.get_series(series_id, observation_start=start_date, observation_end=start_date)
        return data[start_date]

    def yield_curve_nelson_siegel(self):
        treasuries_ids = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3', 'DGS5', \
              'DGS7', 'DGS10', 'DGS20', 'DGS30']
        mat = np.array([[0.0833, 0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30]])
        
        yields_dict = {t_id : float(Interest_Rates._get_yield_data(t_id)) for t_id in treasuries_ids}
        rate_values = list(yields_dict.values())
        y = np.array(rate_values)
        print(mat.shape, y.shape)
        yield_curve = calibrate_ns_ols(mat,y,tau0=1.0)
        return yields_dict
    

    



    

@dataclass
class Convertible:
    Face_Value: float
    Coupon: float
    Coupon_Frequency : str
    Conversion_Ratio : float
    Conversion_Strike : float 

    First_Coupon_Date : str
    Maturity: str
    Redemption : float
    Underlying_stock_ticker : str
    Day_Count_Convention : Optional[str] = 'ACT/ACT'
    Bond_Currency : Optional[str] = 'EUR'
    Stock_Currency : Optional[str] = 'EUR'
    AI_on_Conversion: Optional[bool] = True

    def conversion_price(self):
        return self.Face_Value/self.Conversion_Ratio
    
    def parity(self):
        #Equivalent value in shares if converted right now
        return None

@dataclass
class Option:
    Underlying: str
    K : float
    T : str  
    r = yf.Ticker('e')


In [79]:
rate_field = Interest_Rates('2026-11-30','Annually').yield_curve_nelson_siegel()

(1, 11) (11,)


AssertionError: Mismatching shapes of time and values

In [75]:
rate_values = list(rate_field.values())

In [77]:
np.array(rate_values).shape

(11,)